#  Corn Diseases Detection - Edge Models Training

**Entrenamiento de 4 arquitecturas edge en Google Colab con GPU**

Este notebook entrena y compara:
- MobileNetV3Large
- EfficientNetLiteB2
- MobileViT (con fine-tuning)
- PMVT (con fine-tuning)

---

##  Configuración Inicial

**IMPORTANTE:** Antes de ejecutar:
1. Runtime > Change runtime type > **GPU (T4)**
2. Sube tu carpeta `data/` a Google Drive en: `MyDrive/corn-diseases-data/`
3. Ejecuta todas las celdas en orden

## 1️⃣ Verificar GPU y Recursos

In [ ]:
# Verificar GPU disponible
!nvidia-smi

import tensorflow as tf
print(f"\nOK TensorFlow version: {tf.__version__}")
print(f"OK GPU disponible: {tf.config.list_physical_devices('GPU')}")

# Verificar memoria
!free -h

## 2️⃣ Clonar Repositorio

In [ ]:
# Clonar repositorio
!git clone https://github.com/ojgonzalezz/corn-diseases-detection.git
%cd corn-diseases-detection

# Verificar estructura
!ls -la

## 3️⃣ Instalar Dependencias

In [ ]:
# Instalar dependencias
!pip install -q -r requirements.txt

# Verificar instalación
import tensorflow as tf
import keras
import mlflow
import numpy as np
import pandas as pd

print("OK Todas las dependencias instaladas correctamente")
print(f"   TensorFlow: {tf.__version__}")
print(f"   Keras: {keras.__version__}")
print(f"   MLflow: {mlflow.__version__}")

## 4️⃣ Montar Google Drive y Copiar Datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("\nOK Google Drive montado")
print("\nVerificando datos en Drive...")
!ls -la /content/drive/MyDrive/corn-diseases-data/

In [ ]:
# Copiar datos desde Google Drive
import shutil
import os

# Ruta en Google Drive (AJUSTA SEGÚN TU ESTRUCTURA)
drive_data_path = '/content/drive/MyDrive/corn-diseases-data'

# Copiar train, val, test
for split in ['train', 'val', 'test']:
    src = f'{drive_data_path}/{split}'
    dst = f'data/{split}'
    
    if os.path.exists(src):
        if os.path.exists(dst):
            shutil.rmtree(dst)
        shutil.copytree(src, dst)
        print(f"OK Copiado: {split}")
    else:
        print(f"⚠️  No encontrado: {src}")

# Verificar estructura
print("\n Estructura de datos:")
!tree data/ -L 2 || find data/ -maxdepth 2 -type d

## 5️⃣ Verificar Datos

In [ ]:
import os

def count_images(path):
    """Cuenta imágenes por clase."""
    counts = {}
    for class_name in os.listdir(path):
        class_path = os.path.join(path, class_name)
        if os.path.isdir(class_path):
            counts[class_name] = len([f for f in os.listdir(class_path) 
                                      if f.endswith(('.jpg', '.jpeg', '.png'))])
    return counts

print(" Distribución de datos:\n")
for split in ['train', 'val', 'test']:
    path = f'data/{split}'
    if os.path.exists(path):
        counts = count_images(path)
        total = sum(counts.values())
        print(f"{split.upper()}: {total} imágenes")
        for class_name, count in counts.items():
            print(f"  - {class_name}: {count}")
        print()
    else:
        print(f"⚠️  {split.upper()}: No encontrado\n")

## 6️⃣ Configurar Variables de Entorno

In [ ]:
# Configurar variables de entorno para TensorFlow
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['PYTHONUNBUFFERED'] = '1'

# Configurar GPU memory growth
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"OK GPU memory growth habilitado para {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(e)

print("OK Configuración de entorno completada")

## 7️⃣ Entrenar Todos los Modelos Edge

**Esto tomará aproximadamente 2-3 horas con GPU T4**

In [ ]:
# Ejecutar entrenamiento de todos los modelos
!python experiments/edge_models/train_all_models.py

## 8️⃣ Comparar Resultados

In [ ]:
# Comparar modelos
!python experiments/edge_models/compare_models.py

## 9️⃣ Seleccionar Mejor Modelo

In [ ]:
# Seleccionar mejor modelo
!python experiments/edge_models/select_best_model.py

## 🔟 Ver Resultados

In [ ]:
# Mostrar archivo de mejor modelo
import json

with open('experiments/edge_models/best_edge_model.json', 'r') as f:
    best_model = json.load(f)

print("🏆 MEJOR MODELO SELECCIONADO\n")
print(f"Modelo: {best_model['selected_model']['name']}")
print(f"\nMétricas:")
print(f"  Accuracy: {best_model['performance_metrics']['test_accuracy']:.2%}")
print(f"  Min Recall: {best_model['performance_metrics']['min_recall']:.2%}")
print(f"\nCaracterísticas:")
print(f"  Tamaño: {best_model['model_characteristics']['model_size_mb']} MB")
print(f"  Parámetros: {best_model['model_characteristics']['total_parameters']:,}")
print(f"  Edge-ready: {'OK Sí' if best_model['model_characteristics']['suitable_for_edge'] else 'ERROR No'}")

# Mostrar comparación completa
import pandas as pd
df_comparison = pd.DataFrame(best_model['all_models_comparison'])
print("\n COMPARACIÓN DE TODOS LOS MODELOS\n")
print(df_comparison.to_string(index=False))

## 1️⃣1️⃣ Descargar Resultados

In [ ]:
# Comprimir resultados para descargar
!zip -r edge_models_results.zip \
    experiments/edge_models/best_edge_model.json \
    experiments/edge_models/comparison_results.csv \
    models/exported/*.keras \
    models/exported/*.json \
    models/mlruns/

print("\nOK Resultados comprimidos en: edge_models_results.zip")
print("\nPara descargar:")
print("  1. Click en el icono de carpeta (Files) a la izquierda")
print("  2. Busca 'edge_models_results.zip'")
print("  3. Click derecho > Download")

# También copiar a Google Drive
!cp edge_models_results.zip /content/drive/MyDrive/
print("\nOK También guardado en Google Drive")

## 1️⃣2️⃣ Ver Experimentos en MLflow (Opcional)

In [ ]:
# Iniciar MLflow UI en Colab
import subprocess
import time

# Iniciar MLflow en background
mlflow_process = subprocess.Popen(
    ['mlflow', 'ui', '--host', '0.0.0.0', '--port', '5000', 
     '--backend-store-uri', 'file:///content/corn-diseases-detection/models/mlruns'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

time.sleep(5)

# Crear túnel público con ngrok (opcional)
print("Para ver MLflow UI:")
print("  1. Instala ngrok: !pip install pyngrok")
print("  2. Ejecuta: from pyngrok import ngrok; ngrok.connect(5000)")
print("\nO descarga los resultados y visualiza en local")

## 1️⃣3️⃣ Limpiar (Opcional)

In [ ]:
# Liberar espacio si es necesario
# !rm -rf data/  # Eliminar datos (ya están en Drive)
# !rm -rf models/mlruns/  # Eliminar runs de MLflow (ya descargados)

print(" Descomenta las líneas anteriores si necesitas liberar espacio")

---

## OK Resumen

**Archivos generados:**
- `experiments/edge_models/best_edge_model.json` - Mejor modelo seleccionado
- `experiments/edge_models/comparison_results.csv` - Comparación completa
- `models/exported/*.keras` - Modelos entrenados
- `models/mlruns/` - Experimentos MLflow
- `edge_models_results.zip` - Todo comprimido

**Siguiente paso:**
1. Descarga `edge_models_results.zip`
2. Extrae en tu proyecto local
3. Usa el mejor modelo para deployment

---

**Desarrollado con ❤️ para Edge Computing**